***
## Import the Required Python Packages and Methods

In [1]:
# Import the required packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
import seaborn as sns
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from IPython.display import IFrame

py.offline.init_notebook_mode(connected=True)

***
## Finding and Evaluating Historical Sugarcane Production Data
The United States Department of Agriculture's (USDA) National Agricultural Statistics Service (NASS) has a good website where I acquired this data using their online searchable database:

https://quickstats.nass.usda.gov/

I selected the categorical data parameters in the following order to obtain the relavant sugarcane data:
    1. Sector:           CROPS
    2. Group:            FIELD CROPS
    3. Commodity:        SUGARCANE
    4. Category:         [PRICE RECIEVED, PRODUCTION]
    5. Data Item:        [SUGARCANE, SUGAR - PRICE RECEIVED, MEASURED IN $/TON,
                          SUGARCANE, SUGAR - PRODUCTION, MEASURED IN $,
                          SUGARCANE, SUGAR - PRODCUTION, MEASURED IN TONS]
    6. Domain:           TOTAL
    7. Geographic Level: STATE
    8. State:            [FLORIDA, HAWAII, LOUISIANA, TEXAS]
    9. Year:             [1909 to 2017]

I received a .csv file via a download link from the website.

***
## Performing Data Integrity Checks
After downloading the .csv file, I then performed the following data integrity checks using pandas DataFrames as well as plotly interactive subplots.

In [2]:
# Import the historical sugarcane data
df_raw_data = pd.read_csv("NASS_USDA/FFFD6900-27F3-3E24-A627-FC347BA73EAE.csv", header=0)
df_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 21 columns):
Program             701 non-null object
Year                701 non-null int64
Period              701 non-null object
Week Ending         0 non-null float64
Geo Level           701 non-null object
State               701 non-null object
State ANSI          701 non-null int64
Ag District         0 non-null float64
Ag District Code    0 non-null float64
County              0 non-null float64
County ANSI         0 non-null float64
Zip Code            0 non-null float64
Region              0 non-null float64
watershed_code      701 non-null int64
Watershed           0 non-null float64
Commodity           701 non-null object
Data Item           701 non-null object
Domain              701 non-null object
Domain Category     701 non-null object
Value               701 non-null object
CV (%)              4 non-null object
dtypes: float64(8), int64(3), object(10)
memory usage: 115.1+ KB


In [3]:
# Reshape the historical sugarcane data into an Annual Production Weight (US ton), Annual 
# Production Value (US $), and Annual Price Received Index ($/ton) by State DataFrame 
# for further data integrity checking
df_sugarcane = df_raw_data[["Program", "Year", "Period", "State", "Data Item", "Value"]] \
            [(df_raw_data["Program"] == "SURVEY") & \
             ((df_raw_data["Period"] == "YEAR") | (df_raw_data["Period"] == "MARKETING YEAR"))] \
            .sort_values(["Year", "State"]).reset_index(drop=True)
        
def valueFormat(row):
    if (row["Data Item"] == "SUGARCANE, SUGAR - PRODUCTION, MEASURED IN TONS") or \
       (row["Data Item"] == "SUGARCANE, SUGAR - PRODUCTION, MEASURED IN $"):
        return int(row.Value.replace(',', ''))
    elif (row["Data Item"] == "SUGARCANE, SUGAR - PRICE RECEIVED, MEASURED IN $ / TON"):
        return float(row.Value.replace(',', ''))

df_sugarcane["Value2"] = df_sugarcane.apply(valueFormat, axis=1)
df_sugarcane = df_sugarcane.pivot_table(index="Year", columns=["State", "Data Item"], values="Value2")
df_sugarcane = df_sugarcane.rename(level="Data Item", columns= \
                            {"SUGARCANE, SUGAR - PRICE RECEIVED, MEASURED IN $ / TON": "PRIndex",
                             "SUGARCANE, SUGAR - PRODUCTION, MEASURED IN $": "Value",
                             "SUGARCANE, SUGAR - PRODUCTION, MEASURED IN TONS": "Weight"})
df_sugarcane = df_sugarcane.rename(level="State", columns={"FLORIDA": "FL", "HAWAII": "HI",
                                                           "LOUISIANA": "LA","TEXAS": "TX"})

In [4]:
# Generate an interactive plot using the plotly package to visually inspect data integrity
trace1 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.FL.Weight, mode='lines+markers',
                    line=dict(color="rgb(35,122,181)"), name="FL, Weight")
trace2 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.HI.Weight, mode='lines+markers',
                    line=dict(color="rgb(255,127,14)"), name="HI, Weight")
trace3 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.LA.Weight, mode='lines+markers',
                    line=dict(color="rgb(44,160,44)"), name="LA, Weight")
trace4 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.TX.Weight, mode='lines+markers',
                    line=dict(color="rgb(214,39,40)"), name="TX, Weight")
trace5 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.FL.Value, mode='lines+markers',
                    line=dict(color="rgb(35,122,181)"), name="FL, Value")
trace6 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.HI.Value, mode='lines+markers',
                    line=dict(color="rgb(255,127,14)"), name="HI, Value")
trace7 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.LA.Value, mode='lines+markers',
                    line=dict(color="rgb(44,160,44)"), name="LA, Value")
trace8 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.TX.Value, mode='lines+markers',
                    line=dict(color="rgb(214,39,40)"), name="TX, Value")
trace9 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.FL.PRIndex, mode='lines+markers',
                    line=dict(color="rgb(35,122,181)"), name="FL, PRIndex")
trace10 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.HI.PRIndex, mode='lines+markers',
                     line=dict(color="rgb(255,127,14)"), name="HI, PRIndex")
trace11 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.LA.PRIndex, mode='lines+markers',
                     line=dict(color="rgb(44,160,44)"),name="LA, PRIndex")
trace12 = go.Scatter(x=df_sugarcane.index, y=df_sugarcane.TX.PRIndex, mode='lines+markers',
                     line=dict(color="rgb(214,39,40)"), name="TX, PRIndex")

fig = tools.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.01)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3, 1, 1)
fig.append_trace(trace4, 1, 1)
fig.append_trace(trace5, 2, 1)
fig.append_trace(trace6, 2, 1)
fig.append_trace(trace7, 2, 1)
fig.append_trace(trace8, 2, 1)
fig.append_trace(trace9, 3, 1)
fig.append_trace(trace10, 3, 1)
fig.append_trace(trace11, 3, 1)
fig.append_trace(trace12, 3, 1)

fig["layout"].update(
    plot_bgcolor="rgb(247,247,247)",
    legend=dict(font=dict(family="serif", size=12)),
    height=1000,
    title="<b>Historical U.S. Sugarcane Production and Price Receieved Index</b>",
    titlefont=dict(family="serif", size=24),
    yaxis1=dict(title="<b>Production, US ton</b>", titlefont=dict(family="serif", size=14),
                tickfont=dict(family="serif", size=14)),
    yaxis2=dict(title="<b>Production, US $</b>", titlefont=dict(family="serif", size=14),
                tickfont=dict(family="serif", size=14)),
    yaxis3=dict(title="<b>Price Received Index, $/ton</b>", titlefont=dict(family="serif", size=14),
                tickfont=dict(family="serif", size=14)),
    xaxis1=dict(title="<b>Year</b>", titlefont=dict(family="serif", size=14),
                tickfont=dict(family="serif", size=14)))

py.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]



## Historical Sugarcane Production by Weight Data Evaluation Notes
Looking at the above plotly interactive scatter plot, I notice a few things:
1. Texas has a large gap spanning from 1923 to 1973 in its production by weight data
2. Hawaii has a downward production trend starting in 1968 that may not be attributed to weather conditions alone
3. Lousiana has the longest span of continuous production by weight data ranging from 1909 to 2017
4. Florida has a continuous production by weight data span ranging from 1928 to 2017
5. Texas' production by weight has held relatively constant from 1973 to 2017
6. Florida's and Louisiana's peaks and dips relatively track together from 1996 to 2017

## Historical Sugarcane Production by Value Data Evaluation Notes
Looking at the above plotly interactive scatter plot, I notice a few things:
1. The time span (1978 to 2016) does not match the production by weight data's time span (1909 to 2017)
2. Again, Hawaii has a downward production trend that may not be attributed to weather conditions alone
3. Again, Texas' sugarcane production has held relatively constant

## Historical Sugarcane Price Received Index Data Evaluation Notes
Looking at the above plotly interactive scatter plot, I notice a few things:
1. The time span (1978 to 2016) does not match the production by weight data's time span (1909 to 2017)
2. These price received indexes start to relatively spearate and become volatile in 2009
***

## Export the Reshaped DataFrame for Future Exploratory Data Analysis
Save the manipulated annual historical sugarcane production data as a .csv file for future use inside another Jupyter Notebook.

In [6]:
df_sugarcane.to_csv("df_sugarcane.csv")
df_sugarcane.head()

State          FL                   HI                   LA                   \
Data Item PRIndex Value Weight PRIndex Value Weight PRIndex Value     Weight   
Year                                                                           
1909          NaN   NaN    NaN     NaN   NaN    NaN     NaN   NaN  4229000.0   
1910          NaN   NaN    NaN     NaN   NaN    NaN     NaN   NaN  5046000.0   
1911          NaN   NaN    NaN     NaN   NaN    NaN     NaN   NaN  5887000.0   
1912          NaN   NaN    NaN     NaN   NaN    NaN     NaN   NaN  2163000.0   
1913          NaN   NaN    NaN     NaN   NaN    NaN     NaN   NaN  4214000.0   

State          TX                  
Data Item PRIndex Value    Weight  
Year                               
1909          NaN   NaN  160000.0  
1910          NaN   NaN  176000.0  
1911          NaN   NaN  123000.0  
1912          NaN   NaN  129000.0  
1913          NaN   NaN  112000.0